### Set up

#### 1. Set  up  accounts and role

In [1]:
#!pip install sagemaker==1.39.0

In [2]:
import sagemaker
import boto3

sagemaker_session = sagemaker.Session()
account_id =  boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name


#role = sagemaker.get_execution_role()
role="arn:aws:iam::{}:role/service-role/AmazonSageMaker-ExecutionRole-20190118T115449".format(account_id)


#### 2. Setup image and instance type

In [3]:
pytorch_custom_image_name="ppi-extractor:gpu-1.0.0-201910130520"
instance_type = "ml.p3.8xlarge" 

In [4]:
docker_repo = "{}.dkr.ecr.{}.amazonaws.com/{}".format(account_id, region, pytorch_custom_image_name)

#### 3. Configure train/ test and validation datasets

In [5]:
bucket = "aegovan-data"

In [6]:
# trainfile = "s3://{}/processed_dataset/train_unique_pub_v6_less_negative.json".format(bucket)
# testfile= "s3://{}/processed_dataset/test_unique_pub_v6_less_negative.json".format(bucket)
# valfile="s3://{}/processed_dataset/val_unique_pub_v6_less_negative.json".format(bucket)


trainfile = "s3://{}/processed_dataset/train_unique_pub_v6_max_neg.json".format(bucket)
testfile= "s3://{}/processed_dataset/test_unique_pub_v6_max_neg.json".format(bucket)
valfile="s3://{}/processed_dataset/val_unique_pub_v6_max_neg.json".format(bucket)
# trainfile = "s3://{}/aimed/train_unique_negative_entity_only.json".format(bucket)
# testfile= "s3://{}/aimed/test_unique_negative_entity_only.json".format(bucket)
# valfile= "s3://{}/aimed/val_unique_negative_entity_only.json".format(bucket)


trainfile = "s3://{}/processed_dataset/train_multiclass.json".format(bucket)
testfile= "s3://{}/processed_dataset/test_multiclass.json".format(bucket)
valfile="s3://{}/processed_dataset/val_multiclass.json".format(bucket)

pretrained_bert="s3://{}/embeddings/bert/".format(bucket)


s3_output_path= "s3://{}/results/".format(bucket)
s3_code_path= "s3://{}/aimed_bert_code".format(bucket)

### Start training

In [7]:
pub_inputs = {
    "train" : trainfile,
    "val" :valfile,
    "test" : testfile,
    "PRETRAINED_BIOBERT" : pretrained_bert
}



In [8]:
BertNetworkFactoryhyperparameters = {
    "dataset":"PpiDatasetFactory",
    "network" :"RelationExtractorBioBertFactory",
    "trainfile":trainfile.split("/")[-1],
     "valfile":valfile.split("/")[-1],
    "testfile":testfile.split("/")[-1],
    "batchsize": "8",
    "accumulation_steps": 8,
    "epochs" : "1000",   
    "log-level" : "INFO",
    "learningrate":.00001,
    
    "earlystoppingpatience":20
}

In [9]:
BertNetworkFactoryhyperparameters_multiclass = {
    "dataset":"PpiMulticlassDatasetFactory",
    "network" :"RelationExtractorBioBertFactory",
    "trainfile":trainfile.split("/")[-1],
     "valfile":valfile.split("/")[-1],
    "testfile":testfile.split("/")[-1],
    "batchsize": "8",
    "accumulation_steps": 8,
    "epochs" : "1000",   
    "log-level" : "INFO",
    "learningrate":.00001,
    
    "earlystoppingpatience":20
}

In [10]:
BertNoType_NetworkFactoryhyperparameters = {
    "dataset":"PpiNoInteractionDatasetFactory",
    "network" :"RelationExtractorBioBertFactory",
    "trainfile":trainfile.split("/")[-1],
     "valfile":valfile.split("/")[-1],
    "batchsize": "8",
    "epochs" : "1000",   
    "log-level" : "INFO",
    "learningrate":.00001,
    "earlystoppingpatience":20
}

In [11]:
metric_definitions = [{"Name": "TrainLoss",
                     "Regex": "###score: train_loss### (\d*[.]?\d*)"}
                    ,{"Name": "ValidationLoss",
                     "Regex": "###score: val_loss### (\d*[.]?\d*)"}
                    ,{"Name": "TrainFScore",
                     "Regex": "###score: train_fscore### (\d*[.]?\d*)"}
                   ,{"Name": "ValidationFScore",
                     "Regex": "###score: val_fscore### (\d*[.]?\d*)"}
                    ]

In [12]:
!git log -1 | head -1
!git log -1 | head -5 | tail -1

commit 7e6e36a89f9d7bd04a37eeb853c024879eb581f1
    Fix confusion matric


In [13]:
git_config = {'repo': 'https://github.com/elangovana/PPI-typed-relation-extractor.git',
              'branch': 'master',
            #  'commit': '58a09e154935248667062a36fdae7d86b86b477c'
             }

In [14]:
hyperparameters =BertNetworkFactoryhyperparameters_multiclass #BertNoType_NetworkFactoryhyperparameters 
inputs = pub_inputs 


In [15]:
hyperparameters

{'accumulation_steps': 8,
 'batchsize': '8',
 'dataset': 'PpiMulticlassDatasetFactory',
 'earlystoppingpatience': 20,
 'epochs': '1000',
 'learningrate': 1e-05,
 'log-level': 'INFO',
 'network': 'RelationExtractorBioBertFactory',
 'testfile': 'test_multiclass.json',
 'trainfile': 'train_multiclass.json',
 'valfile': 'val_multiclass.json'}

In [16]:
inputs

{'PRETRAINED_BIOBERT': 's3://aegovan-data/embeddings/bert/',
 'test': 's3://aegovan-data/processed_dataset/test_multiclass.json',
 'train': 's3://aegovan-data/processed_dataset/train_multiclass.json',
 'val': 's3://aegovan-data/processed_dataset/val_multiclass.json'}

In [17]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
     #entry_point='main_train_k_fold.py',
    entry_point='main_train_bert.py',
                    source_dir = 'source/algorithms',
                    dependencies =['source/algorithms', 'source/datasets', 'source/preprocessor', 'source/modelnetworks','source/trainpipelinesbuilders'],
                    role=role,
                    framework_version ="1.0.0",
                    py_version='py3',
                    git_config= git_config,
                    image_name= docker_repo,
                    train_instance_count=1,
                    train_instance_type=instance_type,
                    hyperparameters =hyperparameters,
                    output_path=s3_output_path,
                    metric_definitions=metric_definitions,
                    #train_use_spot_instances = True
                    train_volume_size=30,
                    code_location=s3_code_path,
                    base_job_name ="ppi-bert-extractor-neg")

In [ ]:
estimator.fit(inputs)

2019-11-02 08:09:10 Starting - Starting the training job...
2019-11-02 08:09:12 Starting - Launching requested ML instances...
2019-11-02 08:10:06 Starting - Preparing the instances for training......
2019-11-02 08:11:21 Downloading - Downloading input data
2019-11-02 08:11:21 Training - Downloading the training image.........
2019-11-02 08:13:19 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2019-11-02 08:13:20,871 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2019-11-02 08:13:20,915 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2019-11-02 08:13:22,331 sagemaker_pytorch_container.training INFO     Invoking user training script.
2019-11-02 08:13:22,648 sagemaker-containers INFO     Module main_train_bert does not provide a setup.py. 
Generating setup.py
2019-11-02 08:13:22,6

2019-11-02 08:13:46,988 - algorithms.transform_berttext_tokenise - INFO - Unknown tokens count 0
2019-11-02 08:13:46,989 - algorithms.transform_berttext_tokenise - INFO - Completed TransformBertTextTokenise
2019-11-02 08:13:46,989 - algorithms.transform_berttext_token_to_index - INFO - Transforming TransformBertTextTokenToIndex
2019-11-02 08:13:47,371 - algorithms.transform_berttext_token_to_index - INFO - Completed TransformBertTextTokenToIndex
2019-11-02 08:13:47,391 - algorithms.transform_berttext_tokenise - INFO - Transforming TransformBertTextTokenise
2019-11-02 08:13:49,276 - algorithms.transform_berttext_tokenise - INFO - Unknown tokens count 0
2019-11-02 08:13:49,276 - algorithms.transform_berttext_tokenise - INFO - Completed TransformBertTextTokenise
2019-11-02 08:13:49,276 - algorithms.transform_berttext_token_to_index - INFO - Transforming TransformBertTextTokenToIndex
2019-11-02 08:13:49,323 - algorithms.transform_berttext_token_to_index - INFO - Completed TransformBertText

2019-11-02 08:24:33,440 - algorithms.BertTrain - INFO - Train set result details:
2019-11-02 08:24:33,445 - algorithms.result_writer - INFO - Confusion matrix, full output in /opt/ml/output/data/predictedvsactual_73f2fe7b-58bf-4419-a20b-773f71aefd5b_20191102_082433.csv: 
[[   0    0    0    0    0    4   25    0]
 [   0    0    0    0    0    0    4    0]
 [   0    0    0    0    0   35   73    0]
 [   0    0    0    0    0    1   11    0]
 [   0    0    0    0    0    6   42    0]
 [   0    0    0    0    0 2235  149    0]
 [   0    0    0    0    0  103  526    0]
 [   0    0    0    0    0    5    4    0]]
2019-11-02 08:24:33,449 - algorithms.BertTrain - INFO - Train set result details: 0.20694853956787862
2019-11-02 08:24:33,449 - algorithms.BertTrain - INFO - Validation set result details:
2019-11-02 08:24:38,441 - algorithms.result_writer - INFO - Confusion matrix, full output in /opt/ml/output/data/predictedvsactual_e6d46b40-4b3d-4c4a-9e4e-faf8744f18a9_20191102_082438.csv: 
[[  